In [1]:
# Install optuna if needed
# !pip install optuna loguru

In [2]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)
import warnings
import os

warnings.filterwarnings('ignore')

# Add src to path
import sys
sys.path.append('..')

from src.config import (
    SEQUENCES_DIR, LOGS_DIR, OPTUNA_DB_PATH,
    OUTPUT_SEQ_LEN, OPTUNA_SEARCH_SPACE, OPTUNA_N_TRIALS, OPTUNA_TIMEOUT,
    DEVICE, RANDOM_SEED, set_seed
)
from src.optuna_optimization import run_optuna_optimization
from src.utils import load_json, save_json

# Set seed
set_seed(RANDOM_SEED)

print("Libraries imported successfully!")
print(f"Device: {DEVICE}")

Libraries imported successfully!
Device: cuda


## 6a.1 Load Data

In [3]:
# Load sequences
X_train = np.load(os.path.join(SEQUENCES_DIR, 'X_train.npy'))
y_train = np.load(os.path.join(SEQUENCES_DIR, 'y_train.npy'))
X_val = np.load(os.path.join(SEQUENCES_DIR, 'X_val.npy'))
y_val = np.load(os.path.join(SEQUENCES_DIR, 'y_val.npy'))

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")

X_train: (19394, 24, 22)
y_train: (19394, 5)
X_val: (4156, 24, 22)
y_val: (4156, 5)


## 6a.2 Define Search Space

In [4]:
# Print search space from config
print("Optuna Search Space:")
print("=" * 50)
for param, values in OPTUNA_SEARCH_SPACE.items():
    if isinstance(values, list):
        print(f"  {param}: categorical {values}")
    else:
        print(f"  {param}: range {values}")
print("=" * 50)
print(f"\nNumber of trials: {OPTUNA_N_TRIALS}")
print(f"Timeout: {OPTUNA_TIMEOUT/3600:.1f} hours")

Optuna Search Space:
  hidden_size: categorical [64, 128, 256]
  num_layers: categorical [1, 2, 3]
  dropout: range (0.1, 0.5)
  learning_rate: range (0.0001, 0.01)
  batch_size: categorical [32, 64, 128]
  weight_decay: range (1e-06, 0.001)
  teacher_forcing_ratio: range (0.3, 0.7)

Number of trials: 50
Timeout: 2.0 hours


In [5]:
# Optional: Custom search space (uncomment to override config)
custom_search_space = {
    'hidden_size': [64, 128, 256],
    'num_layers': [1, 2, 3],
    'dropout': (0.1, 0.5),
    'learning_rate': (1e-4, 1e-2),
    'batch_size': [32, 64, 128],
    'weight_decay': (1e-6, 1e-3),
    'teacher_forcing_ratio': (0.3, 0.7),
}

# Use custom or config search space
search_space = OPTUNA_SEARCH_SPACE  # or custom_search_space

## 6a.3 Run Optimization

In [6]:
# Create logs directory
os.makedirs(LOGS_DIR, exist_ok=True)

# Run Optuna optimization
# Note: This will take a while depending on n_trials and timeout
# Use db_path=None for in-memory storage (avoids file lock issues)
results = run_optuna_optimization(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    output_seq_len=OUTPUT_SEQ_LEN,
    search_space=search_space,
    n_trials=OPTUNA_N_TRIALS,  # Reduce this for faster testing
    timeout=OPTUNA_TIMEOUT,
    device=DEVICE,
    log_dir=LOGS_DIR,
    db_path=None  # Use in-memory storage to avoid file lock issues
)

[I 2025-12-05 04:49:57,080] A new study created in memory with name: seq2seq_20251205_044957



Starting Optuna Optimization
Trials: 50
Timeout: 2.0 hours
Device: cuda



  0%|          | 0/50 [00:00<?, ?it/s]

Model built on cuda
Total parameters: 1,343,361
Trainable parameters: 1,343,361
[I 2025-12-05 04:51:16,592] Trial 0 finished with value: 0.0029377857393298587 and parameters: {'hidden_size': 128, 'num_layers': 3, 'dropout': 0.24037782209878922, 'learning_rate': 0.0013272182702944917, 'batch_size': 32, 'weight_decay': 0.0005338393895415409, 'teacher_forcing_ratio': 0.4872950591987881}. Best is trial 0 with value: 0.0029377857393298587.
Model built on cuda
Total parameters: 344,001
Trainable parameters: 344,001
[I 2025-12-05 04:51:16,592] Trial 0 finished with value: 0.0029377857393298587 and parameters: {'hidden_size': 128, 'num_layers': 3, 'dropout': 0.24037782209878922, 'learning_rate': 0.0013272182702944917, 'batch_size': 32, 'weight_decay': 0.0005338393895415409, 'teacher_forcing_ratio': 0.4872950591987881}. Best is trial 0 with value: 0.0029377857393298587.
Model built on cuda
Total parameters: 344,001
Trainable parameters: 344,001
[I 2025-12-05 04:51:49,842] Trial 1 finished with 

## 6a.4 Results Analysis

In [7]:
# Print best parameters
print("\n" + "=" * 60)
print("OPTIMIZATION RESULTS")
print("=" * 60)
print(f"\nCompleted trials: {results['n_completed_trials']}")
print(f"Pruned trials: {results['n_pruned_trials']}")
print(f"Duration: {results['duration_minutes']:.1f} minutes")
print(f"\nBest trial: #{results['best_trial_number']}")
print(f"Best validation loss: {results['best_value']:.6f}")
print("\nBest hyperparameters:")
for param, value in results['best_params'].items():
    print(f"  {param}: {value}")
print("=" * 60)


OPTIMIZATION RESULTS

Completed trials: 31
Pruned trials: 19
Duration: 46.3 minutes

Best trial: #28
Best validation loss: 0.002645

Best hyperparameters:
  hidden_size: 256
  num_layers: 1
  dropout: 0.3404530882152753
  learning_rate: 0.0009042773242170663
  batch_size: 32
  weight_decay: 4.853856662660514e-05
  teacher_forcing_ratio: 0.5950107465643104


In [8]:
# Save best parameters
best_params_path = os.path.join(LOGS_DIR, 'best_params.json')
save_json(results['best_params'], best_params_path)
print(f"Best parameters saved to: {best_params_path}")

Saved: d:\DeepLearning_final\logs\best_params.json
Best parameters saved to: d:\DeepLearning_final\logs\best_params.json


## 6a.5 Visualize Optimization

In [9]:
# Get study object
study = results['study']

In [10]:
# 1. Optimization history
fig = plot_optimization_history(study)
fig.update_layout(title="Optimization History")
fig.show()

In [11]:
# 2. Parameter importance
try:
    fig = plot_param_importances(study)
    fig.update_layout(title="Hyperparameter Importance")
    fig.show()
except:
    print("Not enough trials to compute parameter importance")

In [12]:
# 3. Parallel coordinate plot
fig = plot_parallel_coordinate(study)
fig.update_layout(title="Parallel Coordinate Plot")
fig.show()

In [13]:
# 4. Slice plot for each parameter
fig = plot_slice(study)
fig.show()

In [14]:
# 5. Manual visualization - Trials dataframe
import pandas as pd

trials_df = study.trials_dataframe()
print("Top 10 trials:")
display(trials_df.nsmallest(10, 'value')[['number', 'value', 'state'] + 
        [col for col in trials_df.columns if col.startswith('params_')]])

Top 10 trials:


,number,value,state,params_batch_size,params_dropout,params_hidden_size,params_learning_rate,params_num_layers,params_teacher_forcing_ratio,params_weight_decay
28,28,0.002645,COMPLETE,32,0.340453,256,0.000904,1,0.595011,0.000049
18,18,0.002652,COMPLETE,32,0.159763,256,0.000549,3,0.636065,0.000013
30,30,0.002655,COMPLETE,32,0.408176,256,0.000381,1,0.582725,0.000056
20,20,0.002672,COMPLETE,32,0.102026,256,0.000536,1,0.689924,0.000005
22,22,0.002689,COMPLETE,32,0.168451,256,0.001472,1,0.656161,0.000019
40,40,0.002698,COMPLETE,32,0.437828,256,0.000784,3,0.607314,0.000059
23,23,0.002709,COMPLETE,32,0.151427,256,0.000643,1,0.666987,0.000024
16,16,0.002751,COMPLETE,32,0.198628,128,0.001176,3,0.533998,0.000013
32,32,0.002774,COMPLETE,32,0.392331,256,0.000526,1,0.597783,0.000045
42,42,0.002836,COMPLETE,32,0.156471,256,0.000500,1,0.671709,0.000031


## 6a.6 Continue from Previous Study (Optional)

In [15]:
# Load existing study to continue optimization
# study = optuna.load_study(
#     study_name="seq2seq_optimization",
#     storage=f"sqlite:///{OPTUNA_DB_PATH}"
# )
# print(f"Loaded study with {len(study.trials)} trials")
# print(f"Best value so far: {study.best_value}")

## Summary

**Optuna Optimization completed:**
1. ✅ Loaded training and validation data
2. ✅ Defined hyperparameter search space
3. ✅ Ran optimization with logging
4. ✅ Saved best parameters
5. ✅ Visualized optimization results

**Next step:** 
- Use best parameters in `06_Model_Training.ipynb`
- Or load from `logs/best_params.json`